In [305]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
!pip install folium==0.5.0
import folium
print("done")

     |████████████████████████████████| 81kB 3.9MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
done


<b> 1. Extract HTML elements from page to here for preparation, use them to create three lists, each will represent a column in the dataframe</b>

In [272]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
soup = BeautifulSoup(page.content,'html.parser')
tb = soup.find('table',class_="wikitable sortable")
#rows = tb.find_all('tr')
cols= tb.find_all('td')

#====================create first version of each column=================
i=0;
Postcode=[]
Borough=[]
Neighbourhood=[]
while i < len(cols):
    Postcode.append(cols[i])
    Borough.append(cols[i+1])
    Neighbourhood.append(cols[i+2])
    i=i+3

#===========================clean the lists==================================
Postcode_fin=[]
Borough_fin=[]
Neighbourhood_fin=[]
#scan thru the lines in list Postcode, only keep the text in each line 
for code in Postcode:
    Postcode_fin.append(code.get_text())

for code in Borough:
    Borough_fin.append(code.get_text())

for code in Neighbourhood:
    line=code.get_text()
    Neighbourhood_fin.append(line.rsplit('\n')[0])
#================put together the dataframe ================================
dict = {"Postcode":Postcode_fin,"Borough":Borough_fin,"Neighbourhood": Neighbourhood_fin}
df = pd.DataFrame(dict)

<b> 2. Clean the data </b>

In [273]:
#=================look for row location and use df.drop to remove them =====================
i=0
idx=[]
for b in df['Borough']=='Not assigned':
    if b==True:
        idx.append(i)
    i=i+1
df.drop(index=idx, axis = 0, inplace = True)

#=================look for "Not assigned " in "Neighbourhood" and assign them as the "Borough" value=====================
i=0
idx=[]
for b in df['Neighbourhood']=="Not assigned":
    if b==True:
        df.replace(df.iloc[i,2],df.iloc[i,1],inplace=True)
    i=i+1

#===============look for duplicated value in column "Postcode"====================
df.sort_values(by=['Postcode'],inplace=True)
df.reset_index(drop=True, inplace = True)
df_fin=pd.DataFrame({'Postcode':[],'Borough':[],'Neighbourhood':[]})
df_fin.reset_index(drop=True,inplace = True)
df_fin.append(df.iloc[[0,1]],sort = False)
#for i in range(1, len(df)):
i =0
b=i+1
while b<len(df['Postcode']): 
    if df.iloc[i,0]==df.iloc[b,0]:
        df.iloc[i,2]=df.iloc[i,2]+", " + df.iloc[b,2]
        df.drop([b,b],inplace =True)
        df.reset_index(drop=True, inplace = True)
    i=i+1
    b=i+1

<b> Q1: print the number of rows in the df </b>

In [233]:
df.shape

(139, 3)

<b> Q2: create df with latitude and longtitude shown </b>

In [274]:
df_geo = pd.read_csv("https://cocl.us/Geospatial_data")
df_withLL= df.join(df_geo.set_index('Postal Code'),on='Postcode' )
df_withLL

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill",43.784535,-79.160497
2,M1C,Scarborough,Highland Creek,43.784535,-79.160497
3,M1E,Scarborough,"Guildwood, Morningside",43.763573,-79.188711
4,M1E,Scarborough,West Hill,43.763573,-79.188711
5,M1G,Scarborough,Woburn,43.770992,-79.216917
6,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
7,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
8,M1K,Scarborough,"East Birchmount Park, Ionview",43.727929,-79.262029
9,M1K,Scarborough,Kennedy Park,43.727929,-79.262029


<b> Q3: cluster based on borough and show map </b>

In [345]:
#=============================select best k for clustering and assign Borough in the list to the cluster ===============================
df_withLL.dropna(axis=0,how='any',subset=['Latitude','Longitude'],inplace=True)
X=df_withLL.loc[:,['Borough','Latitude','Longitude']]
K_clusters = range(1,10)
kmeans = [KMeans(n_clusters=i) for i in K_clusters]
Y_axis = df_withLL[['Latitude']]
X_axis = df_withLL[['Longitude']]
score = [kmeans[i].fit(Y_axis).score(Y_axis) for i in range(len(kmeans))]
# Visualize
#plt.plot(K_clusters, score)
#plt.xlabel('Number of Clusters')
#plt.ylabel('Score')
#plt.title('Elbow Curve')
#plt.show()
#best_k=3
kmeans = KMeans(n_clusters = 3, init ='k-means++')
kmeans.fit(X[X.columns[1:3]]) # Compute k-means clustering.
X['cluster_label'] = kmeans.fit_predict(X[X.columns[1:3]])
centers = kmeans.cluster_centers_ # Coordinates of cluster centers.
labels = kmeans.predict(X[X.columns[1:3]]) # Labels of each point

#========================start creating map =========================================================
map = traffic_map = folium.Map(location=[43.7181552,-79.5184855], zoom_start=11)
colordict = {0: 'red', 1: 'green', 2: 'orange'}
for lat, lon, bor, label in zip(X['Latitude'], X['Longitude'],X['Borough'],X['cluster_label']):
    folium.Marker(
        location = [lat, lon],
        icon=folium.Icon(color=colordict[label])
        ).add_to(map)
map